In [13]:
import pymysql, talib, sys, os, tempfile
import pandas as pd , numpy as np
#############################################
class HKStock:
    conn = None
    symbol=None
    daily=None
    tmpfilename=None
    ####################
    def __init__(self,symbol):
        self.symbol = symbol
        self.populate()
        self.saveCSV()
    ####################
    def populate(self):
        if self.conn is None:
            self.conn = pymysql.connect(host='tptcn.ddns.net', port=53306, db='p_master',
                                   user=os.environ['MYSQL_READUSER'], password=os.environ['MYSQL_READPASSWORD'],
                                   cursorclass=pymysql.cursors.DictCursor)
        tmpl = "SELECT  * FROM `consolidated_daily` WHERE `symbol`='%s' "
        sql = tmpl % (self.symbol)
        self.daily= pd.read_sql(sql, self.conn, index_col=['Date'] )
        ### temp
        sOpen  = self.daily['Open']
        sHigh  = self.daily['High']
        sLow   = self.daily['Low']
        sClose = self.daily['Close']
        sVolume= self.daily['Volume']
    ####################
    def saveCSV(self):
        self.tmpfilename = '/tmp/%s' %(self.symbol)
        self.daily.to_csv(self.tmpfilename, columns=['Open', 'High', 'Low', 'Close', 'Volume'], index_label='Date Time')

    ####################
    def __del__(self):
        if self.conn is not None:
            self.conn.close()
            self.conn = None
#############################################
st = HKStock('0388.HK')


In [14]:
from pyalgotrade.barfeed.csvfeed import GenericBarFeed
from pyalgotrade.bar import Frequency

feed = GenericBarFeed(Frequency.DAY,maxLen=10000)
feed.setDateTimeFormat('%Y-%m-%d')
feed.addBarsFromCSV(st.symbol, st.tmpfilename)
print feed

Exception: high < open on 2014-03-12 00:00:00